In [14]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
import patchify
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image as skimg
import imgaug as ia
import imgaug.augmenters as iaa
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_V2_Weights
import torch.optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.insert(1, '/home/prakharug/AFO')
sys.path.insert(1, '/home/prakharug/AFO/pycoco')
from pycoco.engine import train_one_epoch, evaluate

In [15]:
instance_toimage = []

for im_name in os.listdir("./dataset1/images"):
    tmplist = []
    lent = len(im_name[:-4])
    print(im_name,im_name[:lent])
    imgtmp = cv2.imread("./dataset1/images/"+im_name,1).transpose(2,0,1)
    #print(imgtmp[0].shape)
    tmplist.append(imgtmp[0])
    tmplist.append(imgtmp[1])
    tmplist.append(imgtmp[2])
    for gt_name in os.listdir("./dataset1/ground_truths1"):
        #print(gt_name[10+lent],gt_name[10:10+lent])
        if gt_name[10:10+lent]==im_name[:lent] and gt_name[10+lent]=="_":
            mask = np.array(cv2.imread("./dataset1/ground_truths1/"+gt_name,0))
            # mask = (mask > 0).astype(np.uint8) 
            tmplist.append(mask)
    tmplist = np.array(tmplist)
    print(tmplist.shape)
    instance_toimage.append(tmplist)

for im_name in os.listdir("./dataset1/test"):
    tmplist = []
    lent = len(im_name[:-4])
    print(im_name,im_name[:lent])
    imgtmp = cv2.imread("./dataset1/test/"+im_name,1).transpose(2,0,1)
    #print(imgtmp[0].shape)
    tmplist.append(imgtmp[0])
    tmplist.append(imgtmp[1])
    tmplist.append(imgtmp[2])
    for gt_name in os.listdir("./dataset1/ground_truths1"):
        #print(gt_name[10+lent],gt_name[10:10+lent])
        if gt_name[10:10+lent]==im_name[:lent] and gt_name[10+lent]=="_":
            mask = np.array(cv2.imread("./dataset1/ground_truths1/"+gt_name,0))
            # mask = (mask > 0).astype(np.uint8) 
            tmplist.append(mask)
    tmplist = np.array(tmplist)
    print(tmplist.shape)
    instance_toimage.append(tmplist)

9458.bmp 9458
(17, 1536, 2040)
2250.bmp 2250
(21, 1536, 2040)
2179.bmp 2179
(4, 1536, 2040)
9599.bmp 9599
(15, 1536, 2040)
1722.bmp 1722
(15, 1536, 2040)
2202.bmp 2202
(12, 1536, 2040)
9362.bmp 9362
(27, 1536, 2040)
1899.bmp 1899
(18, 1536, 2040)
1730.bmp 1730
(22, 1536, 2040)
9455.bmp 9455
(19, 1536, 2040)
2306.bmp 2306
(15, 1536, 2040)
2009.bmp 2009
(8, 1536, 2040)
1743.bmp 1743
(17, 1536, 2040)
1864.bmp 1864
(19, 1536, 2040)
1909.bmp 1909
(32, 1536, 2040)
9295.bmp 9295
(10, 1536, 2040)
214.bmp 214
(19, 1920, 2560)
1894.bmp 1894
(28, 1536, 2040)
2363.bmp 2363
(18, 1536, 2040)
2393.bmp 2393
(29, 1536, 2040)
1934.bmp 1934
(28, 1536, 2040)
2408.bmp 2408
(21, 1536, 2040)
1740.bmp 1740
(11, 1536, 2040)
1818.bmp 1818
(10, 1536, 2040)
9412.bmp 9412
(15, 1536, 2040)
9459.bmp 9459
(21, 1536, 2040)
2149.bmp 2149
(21, 1536, 2040)
216.bmp 216
(12, 1920, 2560)
106.bmp 106
(13, 1920, 2560)
9611.bmp 9611
(13, 1536, 2040)
9573.bmp 9573
(19, 1536, 2040)
1906.bmp 1906
(25, 1536, 2040)
9909.bmp 9909
(8

In [16]:
cell_img = []
j = 1
for mksl in instance_toimage:
    #print(len(mksl))
    for mksin in range(3,len(mksl)):
        x,y,w,h = cv2.boundingRect(mksl[mksin])
        if x*y*w*h==0 or abs(x+w-mksl.shape[2])<=50 or abs(y+h-mksl.shape[1])<=50:
            print("bruh")
            continue
        else:
            stck1 = mksl[0]*(mksl[mksin]/255)
            stck2 = mksl[1]*(mksl[mksin]/255)
            stck3 = mksl[2]*(mksl[mksin]/255)
            print(stck1.shape)
            new_img = np.array([stck1,stck2,stck3],np.uint8)[:,y:y+h,x:x+w]
            print(new_img.shape)
            cell_img.append(new_img)
            #cv2.imwrite('./cell_iso1/jsr'+str(j)+'.png',new_img.transpose(1,2,0))
            j+=1
            #break
            #cv2.waitkey(0)
            #cv2.destroyAllindows

(1536, 2040)
(3, 305, 303)
(1536, 2040)
(3, 274, 244)
(1536, 2040)
(3, 247, 193)
bruh
(1536, 2040)
(3, 231, 202)
(1536, 2040)
(3, 167, 172)
(1536, 2040)
(3, 350, 405)
bruh
bruh
(1536, 2040)
(3, 158, 139)
(1536, 2040)
(3, 222, 205)
(1536, 2040)
(3, 202, 242)
(1536, 2040)
(3, 259, 287)
bruh
(1536, 2040)
(3, 315, 308)
(1536, 2040)
(3, 178, 196)
(1536, 2040)
(3, 371, 336)
(1536, 2040)
(3, 335, 358)
(1536, 2040)
(3, 326, 315)
bruh
(1536, 2040)
(3, 159, 144)
(1536, 2040)
(3, 424, 379)
(1536, 2040)
(3, 276, 317)
(1536, 2040)
(3, 187, 201)
bruh
(1536, 2040)
(3, 327, 304)
bruh
(1536, 2040)
(3, 368, 411)
(1536, 2040)
(3, 211, 190)
bruh
(1536, 2040)
(3, 150, 153)
(1536, 2040)
(3, 208, 212)
bruh
(1536, 2040)
(3, 193, 196)
bruh
(1536, 2040)
(3, 231, 217)
(1536, 2040)
(3, 328, 292)
(1536, 2040)
(3, 183, 178)
(1536, 2040)
(3, 415, 491)
bruh
bruh
bruh
bruh
(1536, 2040)
(3, 156, 205)
(1536, 2040)
(3, 187, 226)
(1536, 2040)
(3, 261, 234)
bruh
(1536, 2040)
(3, 312, 289)
(1536, 2040)
(3, 226, 250)
(1536, 

In [17]:
def blend_non_transparent(face_img, overlay_img):
    # Let's find a mask covering all the non-black (foreground) pixels
    # NB: We need to do this on grayscale version of the image
    #print("overlay img ",overlay_img.shape)
    gray_overlay = cv2.cvtColor(overlay_img, cv2.COLOR_BGR2GRAY)
    overlay_mask = cv2.threshold(gray_overlay, 1, 255, cv2.THRESH_BINARY)[1]

    # Let's shrink and blur it a little to make the transitions smoother...
    #overlay_mask = cv2.erode(overlay_mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))
    #overlay_mask = cv2.blur(overlay_mask, (3, 3))

    # And the inverse mask, that covers all the black (background) pixels
    background_mask = (255) - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [18]:
lob = 1000
for iml in instance_toimage:
    cv2.imwrite("./dataset2.1/images/"+str(lob)+'.png',iml[0:3].transpose(1,2,0))
    for k in range(0,iml.shape[0]-3):
        cv2.imwrite("./dataset2.1/ground_truths/"+str(lob)+'_'+str(k+1)+'.png',np.array([iml[k+3]]).transpose(1,2,0))
    lob+=1
    i=0
    if(iml.shape[0]<=5):
        continue
    while i<=2:
        i+=1
        tmpl = iml.copy()
        cnt=0
        for j in range(0,10):
            paster = False
            idx = random.randint(0,len(cell_img)-1)
            rtrnd=random.randint(0,4)
            flipr = random.randint(0,1)
            flipu = random.randint(0,1)
            seq = iaa.Sequential([
            # iaa.WithHueAndSaturation([
            # iaa.WithChannels(0, iaa.Add((-30, 10))),
            # iaa.WithChannels(1, [
            #     iaa.Multiply((0.5, 1.5)),
            #     iaa.LinearContrast((0.75, 1.25))
            #     ])
            # ])
            # ,
            iaa.Rot90((rtrnd), keep_size=False),
            iaa.Fliplr(flipr),
            iaa.Flipud(flipu),
            ])

            # print(cell_img[idx].shape)
            test = cell_img[idx].transpose(1,2,0)
            cell_paste = seq(images=[test])
            cell_paste = cell_paste[0].transpose(2,0,1)
            #cell_paste = cell_img[idx]
            k_w = cell_paste.shape[2]
            k_h = cell_paste.shape[1]
            tryer=0
            o_w=0
            o_h=0
            while paster==False and tryer<=5:
                tryer+=1
                o_w = random.randint(0,tmpl.shape[2]-k_w)
                o_h = random.randint(0,tmpl.shape[1]-k_h)
                patch_img = tmpl[0:3,o_h:o_h+k_h,o_w:o_w+k_w]
                instances = tmpl[3:,o_h:o_h+k_h,o_w:o_w+k_w]
                instances = instances.transpose(1,2,0)
                for a in range(instances.shape[2]):
                    dispim = instances[:,:,a]
                    #cv2.imwrite('writer.jpg',dispim)
                    if np.all(dispim == 0):
                        paster = True
                        continue
                    else:
                        paster=False
                        break
                    
            if paster==True:
                back = np.zeros([3,tmpl.shape[1],tmpl.shape[2]],np.uint8)
                back[:,o_h:o_h+k_h,o_w:o_w+k_w] = cell_paste
                grao = cv2.cvtColor(back.transpose(1,2,0), cv2.COLOR_BGR2GRAY)
                grao = (grao>0)*255
                grao = np.array([grao])
                #print(cell_paste.shape,k_h)
                #cv2.imwrite('jsr.png',tmpl[0:3].transpose(1,2,0))
                #print(type(tmpl[0:3]),type(back))
                tmpl[0:3] = blend_non_transparent(tmpl[0:3].transpose(1,2,0),back.transpose(1,2,0)).transpose(2,0,1)
                #cv2.imwrite('jsr.png',tmpl[0:3].transpose(1,2,0))
                #cv2.imwrite('comp.png',cell_paste.transpose(1,2,0))
                #print(tmpl.shape,grao.shape)
                tmpl = np.append(tmpl,grao,0)
                #print(tmpl.shape,grao.shape)
                cnt+=1
        if cnt>=3:
            print('hello')
            cv2.imwrite("./dataset2.1/images/"+str(lob)+'.png',tmpl[0:3].transpose(1,2,0))
            for k in range(0,tmpl.shape[0]-3):
                cv2.imwrite("./dataset2.1/ground_truths/"+str(lob)+'_'+str(k+1)+'.png',np.array([tmpl[k+3]]).transpose(1,2,0))
            lob+=1
        else:
            i-=1

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hell

In [19]:
for im_name in os.listdir("./submission/images"):
    tmplist = []
    lent = len(im_name[:-4])
    print(im_name,im_name[:lent])
    imgtmp = cv2.imread("./submission/images/"+im_name,1)
    imgtmp = (imgtmp > 0)*255
    cv2.imwrite("./submission/x/"+im_name[0:-4]+".png",imgtmp)

FileNotFoundError: [Errno 2] No such file or directory: './submission/images'

In [ ]:
for im_name in os.listdir("./test/images2"):
    tmplist = []
    lent = len(im_name[:-4])
    print(im_name,im_name[:lent])
    imgtmp = cv2.imread("./test/images2/"+im_name,1)
    #imgtmp = (imgtmp > 0)*255
    cv2.imwrite("./test/images/"+im_name[0:-4]+".png",imgtmp)

In [ ]:
for im_name in os.listdir("./dataset1/ground_truths1"):
    tmplist = []
    lent = len(im_name[10:-4])
    print(im_name,im_name[10:-4])
    imgtmp = cv2.imread("./dataset1/ground_truths1/"+im_name,1)
    #imgtmp = (imgtmp > 0)*255
    cv2.imwrite("./dataset1/ground_truths/"+im_name[10:-4]+".png",imgtmp)